In [18]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt


In [19]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'


--2024-11-27 20:05:30--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  13.6MB/s    in 1.8s    

2024-11-27 20:05:33 (13.6 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [20]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)


In [ ]:
# add your code here - consider creating a new cell for each section of code

In [21]:
# Data cleaning: removing users with less than 200 ratings and books with less than 100 ratings
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Filter data based on the conditions
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]


In [22]:
# Create a user-item matrix (pivot table)
df_ratings_matrix = df_ratings.pivot(index='user', columns='isbn', values='rating').fillna(0)

# Convert the matrix to a sparse matrix format
sparse_matrix = csr_matrix(df_ratings_matrix.values)

# Fit NearestNeighbors model
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
knn.fit(sparse_matrix)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [23]:
# function to return recommended books
def get_recommends(book = ""):
    # Find the book index for the given book title
    book_index = df_books[df_books['title'] == book].index[0]

    # Get the vector (row) for the selected book from the user-item matrix
    book_vector = sparse_matrix[book_index].reshape(1, -1)

    # Find the nearest neighbors
    distances, indices = knn.kneighbors(book_vector)

    # Prepare the recommended books and their distances
    recommended_books = []
    for i in range(1, 6):  # Skip the first result as it is the same book
        book_title = df_books.iloc[indices[0][i]]['title']
        distance = distances[0][i]
        recommended_books.append([book_title, distance])

    return [book, recommended_books]


In [24]:
# Test function
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

# Test the recommendation system
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(2):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()


["Where the Heart Is (Oprah's Book Club (Paperback))", [['Hannibal', 0.69336796], ['Harlequin Valentine', 0.69918156], ['All the Pretty Horses (Border Trilogy, Vol 1)', 0.705352], ['Wild Animus', 0.7098703], ["The King of Elfland's Daughter (Del Rey Impact)", 0.7579062]]]
You haven't passed yet. Keep trying!
